### Imports

In [77]:

import pandas as pd
import requests, json
import mysql.connector
from sqlalchemy import create_engine
import psycopg2


Making the column names

In [59]:
column_names = ['winning_team','cluster_id','game_mode','game_type']

In [60]:
df = pd.read_csv(r'C:\Users\Owner\Documents\Week 5 Project\dota2Train.csv', usecols=[*range(0,4)],header=None, names=column_names)
df['winning_team'].replace(-1,'dire',inplace=True)
df['winning_team'].replace(1,'radiant',inplace=True)


Pulling from the json of the api to get the cluster id's actual placement

In [61]:
df

,winning_team,cluster_id,game_mode,game_type
0,dire,223,2,2
1,radiant,152,2,2
2,radiant,131,2,2
3,radiant,154,2,2
4,dire,171,2,3
...,...,...,...,...
92645,dire,154,2,3
92646,radiant,154,2,2
92647,radiant,111,2,3
92648,dire,185,2,2


In [62]:
cluster_page = requests.get(r'https://raw.githubusercontent.com/kronusme/dota2-api/master/data/regions.json').json()
cluster_dict={}
for i in range(len(cluster_page['regions'])):
    cluster_dict[str(cluster_page['regions'][i]['id'])]=(cluster_page['regions'][i]['name']).replace(' ','_').lower()
cluster_dict['232']= 'china' # This cluster id is set to the city of Tianjin, China
for i in range(len(df['cluster_id'])):
        df['cluster_id'][i] = cluster_dict[(str(df['cluster_id'][i]))]



C:\Users\Owner\AppData\Local\Temp\ipykernel_13400\1374439600.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['cluster_id'][i] = cluster_dict[(str(df['cluster_id'][i]))]


Setting the game mode

In [63]:
mode_page = requests.get(r'https://raw.githubusercontent.com/kronusme/dota2-api/master/data/mods.json').json()
mode_dict={}
for i in range(len(mode_page['mods'])):
    mode_dict[str(mode_page['mods'][i]['id'])]=(mode_page['mods'][i]['name']).replace(' ','_').lower()
mode_dict['6']= 'intro_death' # directly fixing the ??_*_??
for i in range(len(df['game_mode'])):
        df['game_mode'][i] = mode_dict[(str(df['game_mode'][i]))]


C:\Users\Owner\AppData\Local\Temp\ipykernel_13400\3695662672.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['game_mode'][i] = mode_dict[(str(df['game_mode'][i]))]


Setting the game type

In [64]:
lobby_page = requests.get(r'https://raw.githubusercontent.com/kronusme/dota2-api/master/data/lobbies.json').json()
lobby_dict={}
for i in range(len(lobby_page['lobbies'])):
    lobby_dict[str(lobby_page['lobbies'][i]['id'])]=(lobby_page['lobbies'][i]['name']).replace(' ','_').lower()
for i in range(len(df['game_type'])):
        df['game_type'][i] = lobby_dict[(str(df['game_type'][i]))]

C:\Users\Owner\AppData\Local\Temp\ipykernel_13400\2553531160.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['game_type'][i] = lobby_dict[(str(df['game_type'][i]))]


In [65]:
heros_df = pd.read_csv(r'C:\Users\Owner\Documents\Week 5 Project\dota2Train.csv', usecols=[*range(4,27),*range(28,117)],header=None, na_values=0)
# skipping the column indexed at 27. The entire column is 0 and if were placed at 24 by index
# the i hero id 24 is missing in the dota 2 api as well. No information is provided why.
heros_df = heros_df.transpose().reset_index(drop=True)
hero_page =requests.get(r'https://raw.githubusercontent.com/kronusme/dota2-api/master/data/heroes.json').json()
hero_dict={}
for i in range(len(hero_page['heroes'])):
    # I will be using the index of the heros dataframe to pull the hero name,
    # so I need the index to start at 0.
    if i < 23:
        hero_dict[str((hero_page['heroes'][i]['id']-1))]=(hero_page['heroes'][i]['localized_name']).replace(' ','_').lower()
    # After analyzing, I found #24 missing, so once the ids hit 24, it will
    # subtract to match with the column length of the heroes.
    elif i >=23:
        hero_dict[str((hero_page['heroes'][i]['id'])-2)]=(hero_page['heroes'][i]['localized_name']).replace(' ','_').lower()
# While tracking through the api, discovered it had these two as their original
# Warcraft 3 Dota names, so I changed them to their more relative names.
hero_dict['40'] = 'wraith_king'
hero_dict['89'] = 'io'

# hero_dict
# len(hero_page['heroes'])
# heros_df.isnull().sum().sort_values()
# len(hero_page['heroes'])

# heros_df

In [66]:
# This was to find which id# was missing in the original heroes' dictionary. It was 24.
# for i in range(len(hero_dict)):
#     try:
#         hero_dict[str(i)]
#     except:
        # print(i)
# print(len(hero_dict))
# print(len(heros_df[0]))
# print(len(heros_df.columns.tolist()))
# heros_df[0]

Iterating through the heroes dataframe to pull the hero names to add to the original dataframe

In [67]:
data =[]
for dex in range(len(heros_df.columns.tolist())):
    radiant_team = []
    dire_team = []
    hero_col = {}
    for i in range(len(heros_df[dex])):
        if heros_df[dex][i] == 1:
            radiant_team.append(hero_dict[str(i)])
        elif heros_df[dex][i] == -1:
            dire_team.append(hero_dict[str(i)])
    for ind in range(len(radiant_team)):
        hero_col[f'radiant_player_{ind+1}'] = radiant_team[ind]
    for inde in range(len(dire_team)):
        hero_col[f'dire_player_{inde+1}'] = dire_team[inde]
    data.append(hero_col)
    

In [68]:
df2= pd.DataFrame.from_dict(data)
df3 = pd.concat([df,df2],axis=1)
df3

,winning_team,cluster_id,game_mode,game_type,radiant_player_1,radiant_player_2,radiant_player_3,radiant_player_4,radiant_player_5,dire_player_1,dire_player_2,dire_player_3,dire_player_4,dire_player_5
0,dire,china,captains_mode,tournament,morphling,pudge,lina,slardar,riki,sven,zeus,beastmaster,invoker,nyx_assassin
1,radiant,southeast_asia,captains_mode,tournament,bloodseeker,pudge,lion,shadow_shaman,spirit_breaker,drow_ranger,windranger,sniper,slark,timbersaw
2,radiant,europe_west,captains_mode,tournament,bloodseeker,zeus,lina,riki,huskar,drow_ranger,vengeful_spirit,templar_assassin,gyrocopter,slark
3,radiant,southeast_asia,captains_mode,tournament,storm_spirit,sniper,lifestealer,bounty_hunter,troll_warlord,earthshaker,kunkka,wraith_king,viper,legion_commander
4,dire,australia,captains_mode,tutorial,sand_king,lich,necrophos,phantom_assassin,alchemist,drow_ranger,mirana,shadow_fiend,tidehunter,rubick
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92645,dire,southeast_asia,captains_mode,tutorial,anti-mage,windranger,zeus,witch_doctor,doom,bloodseeker,mirana,dazzle,huskar,invoker
92646,radiant,southeast_asia,captains_mode,tournament,lich,luna,lifestealer,medusa,legion_commander,crystal_maiden,juggernaut,lina,huskar,visage
92647,radiant,us_west,captains_mode,tutorial,mirana,morphling,storm_spirit,riki,undying,faceless_void,luna,bounty_hunter,ancient_apparition,invoker
92648,dire,russia,captains_mode,tournament,drow_ranger,pudge,windranger,lion,phantom_assassin,mirana,phantom_lancer,razor,riki,dragon_knight


Testing the sql server connection

In [78]:
mydb=mysql.connector.connect(user='user_5',password= 'Bonfire124',host= 'localhost',database= 'week_5_project')
cur = mydb.cursor()

# df1 = wrangle(r'C:\Users\Owner\Documents\Week 5 Project\dota2Train.csv')
# df2 = wrangle(r'C:\Users\Owner\Documents\Week 5 Project\dota2Test.csv')
# df = pd.concat([df1,df2],axis=0)


NameError: name 'wrangle' is not defined